# 챗봇 엔진 서버 메인 프로그램

In [1]:
import threading
import sqlite3
import time
import math
import difflib

import import_ipynb   # 다른 ipynb 파일을 import 하기 위해 필요
import SQliteDB
from BotServer import BotServer
from Preprocess import Preprocess
from IntentModel import IntentModel
from NerModel import NerModel
from FindAnswer import FindAnswer
from UserIntent import UserIntent

importing Jupyter notebook from SQliteDB.ipynb
importing Jupyter notebook from BotServer.ipynb
importing Jupyter notebook from Preprocess.ipynb
importing Jupyter notebook from IntentModel.ipynb
importing Jupyter notebook from NerModel.ipynb
importing Jupyter notebook from FindAnswer.ipynb
importing Jupyter notebook from UserIntent.ipynb


## 함수 생성

### 문자-숫자 변환 함수

In [2]:
#변수값들을 DB에서 문자열로된 값을 꺼내오기 때문에 
#1. 예측확률을 계산하려면 변환해줘야 함
#2. 숫자로 제대로 입력이 됐는지 확인해야 함
def convertToNumber(s):
    try:
        return int(s)
    except ValueError:
        try:
            return float(s)
        except ValueError:
            return None

### 모델 생성(일상 대화)

In [3]:
# 전처리 객체 생성
p = Preprocess(word2index_dic='../train_tools/dict/chatbot_dict.bin',
               userdic='../utils/user_dic.tsv')

# 의도 파악 모델
intent = IntentModel(model_name='../models/intent/intent_model.h5', proprocess=p)

# 개체명 인식 모델
ner = NerModel(model_name='../models/ner/ner_model.h5', proprocess=p)


### 공식 추가 함수(공식 추가)

In [4]:
def process_insert_formula(check_input_id, key, answer_mapping):
    formula_db.connect()
    answer1 = answer_mapping[key][1]
    print(f"{local_data.addr} - {answer1}")
    if key == "disease":
        conv_db.save_output_update(local_data.addr, check_input_id, answer1)
    else:
        conv_db.save_output_insert(local_data.addr, answer1)
        
    while True:
        time.sleep(0.5)
        check_input_id = conv_db.check_output_id(local_data.addr, answer1)
        formula_data = conv_db.load_input_data(local_data.addr, check_input_id)
        if formula_data is not None:
            break
    print(f"{local_data.addr} - {key}: {formula_data}")

    while True:
        if not formula_data.strip():
            answer2 = "유효한 값을 입력하세요."
            print(f"{local_data.addr} - {answer2}")
            conv_db.save_output_update(local_data.addr, check_input_id, answer2)

            while True:
                time.sleep(0.5)
                check_input_id = conv_db.check_output_id(local_data.addr, answer2)
                formula_data = conv_db.load_input_data(local_data.addr, check_input_id)
                if formula_data is not None:
                    break
                else:
                    pass
            print(f"{local_data.addr} - {answer_mapping[key][0]}: {formula_data}")

        elif formula_data == "뒤로":
            conv_db.save_output_update(local_data.addr, check_input_id, "공식 추가를 취소합니다.")
            return "stop"
        
        else:
            globals_lock_formula(key, formula_data)
            if key == "disease":
                formula_db.save_disease(formula_data)
            else:
                formula_db.save_formula_data(disease, key, formula_data)

            answer3 = f"{answer_mapping[key][0]} 정보를 저장했습니다."
            print(f"{local_data.addr} - {answer3}")
            conv_db.save_output_update(local_data.addr, check_input_id, answer3)
            time.sleep(1)
            return
    formula_db.close()

#### 공식 변수 지정 함수

In [5]:
def globals_lock_formula(key, data):
    with lock:
        globals()[key] = data
        return globals().get(key, None)

### 사용자 정보 저장(예측 확률)

In [6]:
def process_save_user_data(check_input_id, i, user_id, variable, question_mapping):
    user_db.connect()
    while True:
        user_data = user_db.get_user_info(user_id)[variable][0]
        print(f"{local_data.addr} - DB 정보  {variable}: {user_data}")
        if user_data is None or not user_data.strip():
            answer1 = question_mapping.get(variable, None)['question']
            print(f"{local_data.addr} - {answer1}")
            if i == 1:
                conv_db.save_output_update(local_data.addr, check_input_id, answer1)
            else:
                conv_db.save_output_insert(local_data.addr, answer1)

            while True:
                time.sleep(0.5)
                check_input_id = conv_db.check_output_id(local_data.addr, answer1)
                user_data1 = conv_db.load_input_data(local_data.addr, check_input_id)
                if user_data1 is  not None:
                    break
                else:
                    pass
            print(f"{local_data.addr} - 새로운 정보 1-1  {variable}: {user_data1}")

            while True:
                if not user_data1.strip() or convertToNumber(user_data1) is None:
                    answer2 = "유효한 값을 입력하세요."
                    print(f"{local_data.addr} - {answer2}")
                    conv_db.save_output_update(local_data.addr, check_input_id, answer2)

                    while True:
                        time.sleep(0.5)
                        check_input_id = conv_db.check_output_id(local_data.addr, answer2)
                        user_data1 = conv_db.load_input_data(local_data.addr, check_input_id)
                        if user_data1 is not None:
                            break
                        else:
                            pass
                    print(f"{local_data.addr} - 새로운 정보 1-2  {variable}: {user_data1}")
                else:
                    user_db.save_user_data(user_id, variable, user_data1)
                    globals_lock_var(variable, user_data1)
                    variable_name = question_mapping.get(variable, None)['name']
                    answer3 = f"{variable_name} 정보를 저장했습니다."
                    print(f"{local_data.addr} - {answer3}")
                    conv_db.save_output_update(local_data.addr, check_input_id, answer3)
                return
        else:
            globals_lock_var(variable, user_data)
            variable_name = question_mapping.get(variable, None)['name']
            answer4 = f"{variable_name} 정보 확인 완료"
            print(f"{local_data.addr} - {answer4}")
            if i == 1:
                conv_db.save_output_update(local_data.addr, check_input_id, answer4)
            else:
                conv_db.save_output_insert(local_data.addr, answer4)
            return
    user_db.close()

#### 사용자 변수 지정 함수

In [7]:
def globals_lock_var(variable, data):
    with lock:
        globals()[variable] = convertToNumber(data)
        return globals().get(variable, None)

#### 확률 계산 함수

In [8]:
def cal_risk(load_risk):
    risk_list = []
    for items in load_risk:
        for item in items.split(","):
            try:
                var, form = item.replace(" ","").split("=")
                globals()[var] = eval(form)
                if var == "risk":
                    risk_list.append(eval(form))
            except ValueError:
                print(f"Invalid format: {item}")
    return risk_list

### 사용자 정보 수정 함수(수정)

In [9]:
def process_convert_user_data(check_input_id, user_id, name, question_df):
    user_db.connect()

    question_mapping = question_df.set_index('name')[['variable', 'question']].T.to_dict()
    
    answer1 = question_mapping.get(name, None)['question']
    print(f"{local_data.addr} - {answer1}")
    conv_db.save_output_update(local_data.addr, check_input_id, answer1)

    while True:
        time.sleep(0.5)
        check_input_id = conv_db.check_output_id(local_data.addr, answer1)
        user_data1 = conv_db.load_input_data(local_data.addr, check_input_id)
        if user_data1 is not None:
            break
        else:
            pass
    print(f"{local_data.addr} - 수정한 정보 1-1  {name}: {user_data1}")
    
    while True:
        if not user_data1.strip() or convertToNumber(user_data1) is None:
            answer2 = "유효한 값을 입력하세요."
            print(f"{local_data.addr} - {answer2}")
            conv_db.save_output_update(local_data.addr, check_input_id, answer2)

            while True:
                time.sleep(0.5)
                check_input_id = conv_db.check_output_id(local_data.addr, answer2)
                user_data1 = conv_db.load_input_data(local_data.addr, check_input_id)
                if user_data1 is not None:
                    break
                else:
                    pass
            print(f"{local_data.addr} - 수정한 정보 1-2  {name}: {user_data1}")
        else:
            re_variable = question_mapping.get(name, None)['variable']
            user_db.save_user_data(user_id, re_variable, user_data1)
            globals_lock_var(re_variable, user_data1)
            answer3 = f"{name} 정보를 수정했습니다."
            print(f"{local_data.addr} - {answer3}")
            conv_db.save_output_update(local_data.addr, check_input_id, answer3)
            return
    user_db.close()

### 스레드 함수

In [10]:
def to_client(connect, addr, params, intents_result):
    chat_db = params['chat_db']
    user_db = params['user_db']
    formula_db = params['formula_db']
    question_db = params['question_db']
    disease_db = params['disease_db']
    conv_db = params['conv_db']

    local_data.connect = connect
    local_data.addr = addr
    
    try:
        # DB 연결
        chat_db.connect()    
        conv_db.connect()
        
        # 사용자별 DB 테이블 생성
        conv_db.make_user_table(local_data.addr)
        
        greeting = "안녕하세요, 챗봇입니다. 어떤 도움이 필요하신가요?"
        print(f"{local_data.addr} - {greeting}")
        conv_db.save_output_insert(local_data.addr, greeting)

        while True:
            ret_data = "사용자 ID를 입력하세요."
            print(f"{local_data.addr} - {ret_data}")
            conv_db.save_output_insert(local_data.addr, ret_data)

            while True:
                time.sleep(0.5)
                local_data.check_input_id = conv_db.check_output_id(local_data.addr, ret_data)
                local_data.user_id = conv_db.load_input_data(local_data.addr, local_data.check_input_id)
                if local_data.user_id is not None:
                    break
                else:
                    pass
            print(f"{local_data.addr} - user ID: {local_data.user_id}\n")
            user_db.connect()
            local_data.user_info = user_db.get_user_info(local_data.user_id)

            # 사용자가 바로 창 닫는 경우
            if local_data.user_id == "browser_closed":
                local_data.connect.close()
                local_data.connect = None
                break 
            
            # 사용자 ID 확인
            if local_data.user_info is None:
                # 사용자 ID가 DB에 없는 경우       
                id_confirm1 = '등록되지 않은 사용자입니다.'
                print(f"{local_data.addr} - {id_confirm1}")
                conv_db.save_output_update(local_data.addr, local_data.check_input_id, id_confirm1) 
                continue 
            else:
                id_confirm2 = 'ID 확인 완료'
                print(f"{local_data.addr} - {id_confirm2}")
                conv_db.save_output_update(local_data.addr, local_data.check_input_id, id_confirm2) 
                break

        while True:
            if local_data.connect is None or not local_data.connect:
                # 클라이언트 연결이 끊어지거나, 오류가 있는 경우
                print(f"{local_data.addr} - 클라이언트 연결 끊어짐1")
                break # 반복문 종료       
            
            ret_data = "질문을 입력하세요."
            print(f"{local_data.addr} - {ret_data}")
            conv_db.save_output_insert(local_data.addr, ret_data)

            while True:
                time.sleep(0.5)
                local_data.check_input_id = conv_db.check_output_id(local_data.addr, ret_data)
                local_data.query = conv_db.load_input_data(local_data.addr, local_data.check_input_id)
                if local_data.query is not None:
                    break
                else:
                    pass
            print(f"{local_data.addr} - query: {local_data.query}\n")

            # 사용자가 입력창에 종료 메시지를 입력하여 종료하는 경우
            if local_data.query == "끝" or local_data.query == "종료" :
                print(f"{local_data.addr} - 클라이언트 연결 종료 요청")
                break
            # 사용자가 바로 창 닫는 경우
            if local_data.query == "browser_closed":
                print(f"{local_data.addr} - 클라이언트 연결 끊어짐2")
                break  
                
            # 텍스트 파일에서 질병명과 품사를 읽어옴
            with open("NNG_disease_name.txt", 'r', encoding='utf-8') as file:
                lines = file.readlines()

            question_db.connect()
            question_df = question_db.get_question_info()
            question_db.close()

            # 사용자 입력에서 질병명 추출
            local_data.search_term_list = [disease_name for line in lines for parts in [line.strip().split('\t')] if len(parts) == 2 for disease_name, pos_tag in [parts] if pos_tag == "NNG" and disease_name in local_data.query]
            local_data.search_term = max(local_data.search_term_list, key=lambda term: difflib.SequenceMatcher(None, local_data.query, term).ratio(), default=None)
            print(f"{local_data.addr} - search_term: {local_data.search_term}")
            # 사용자 입력에서 질병명 제외한 문자열로 재생성
            local_data.re_query = local_data.query.replace(local_data.search_term, '').strip() if local_data.search_term else local_data.query
            # 사용자 입력에서 의도 추출
            local_data.user_intent = intents.compare_intent_similarity(local_data.re_query, intents_result)[0]
            print(f"{local_data.addr} - user_intent: {local_data.user_intent}\n")

            # 질병 정보 DB에 있는 띄어쓰기 조합 칼럼으로 질병 찾기
            disease_db.connect()
            if local_data.search_term is None and hasattr(local_data, 'disease_name') and hasattr(local_data, 'disease_info'):
                # 이전 이력에 있는 질병 정보로 구하기
                print(f"{local_data.addr} - disease_name: {local_data.disease_name}, 이전 이력 있음")
            else:
                local_data.result_info = disease_db.get_disease_info(local_data.search_term)
                local_data.disease_name, local_data.disease_info = local_data.result_info
                print(f"{local_data.addr} - disease_name: {local_data.disease_name}\n")
            disease_db.close()
            
            if local_data.user_intent == "공식 추가" and "공식" in local_data.re_query:
                print(f"{local_data.addr} - 질병 공식 추가\n")
                
                answer_mapping = {'disease': ["질병명", "질병명은 무엇인가요?"], 
                            'center':["진료과", "어느 과에 속해있나요?"], 
                            'prediction_formula':["예측 공식", "질병 예측 공식은 무엇인가요?"], 
                            'input_variable':["입력 변수", "공식에 들어갈 입력 변수는 무엇인가요?"],
                          }
                keys = list(answer_mapping.keys())
                i = 0
                while i < len(keys):
                    key = keys[i]
                    print(i, key)
                    result = process_insert_formula(local_data.check_input_id, key, answer_mapping)
                    i += 1
                    if result == "stop":
                        break
                    
            elif local_data.user_intent == "예측 확률":
                print(f"{local_data.addr} - 질병 예측 확률\n")

                formula_db.connect()
                local_data.formula_data = formula_db.get_formula_info(local_data.disease_name)
                formula_db.close()

                # DB에 해당 질병의 공식이 있을 경우
                if local_data.formula_data:
                    local_data.load_var = []
                    local_data.load_risk = []
                    
                    for i, var in local_data.formula_data:
                        if i is not None and var is not None:
                            var_items = var.split(",")
                            local_data.load_var.append([item.replace(" ", "") for item in var_items])
                            local_data.load_risk.append(i)
                                
                    # 여러 질병의 변수이름 합친 후 중복 변수 제거
                    local_data.var_list = []
                    for j in range(len(local_data.load_var)):
                        local_data.var_list.extend(local_data.load_var[j])  
                    local_data.var = sorted(list(set(local_data.var_list)))  
                    local_data.new_var = [item for item in local_data.var if item.strip()]
                    print(f"{local_data.addr} - {len(local_data.new_var)}, {local_data.new_var}")
                    
                    # new_var에 있는 변수들의 질문 리스트 맵핑
                    local_data.question_mapping = question_df.set_index('variable')[['name', 'question']].T.to_dict()
                    
                    # 질문 시작 반복문
                    k = 1
                    while k <= len(local_data.new_var):
                        print(k)
                        local_data.var1 = local_data.new_var[k-1].strip()
                        process_save_user_data(local_data.check_input_id, k, local_data.user_id, local_data.var1, local_data.question_mapping)
                        k += 1
                    print("질문 끝\n")

                    # 위험확률 정보 보내기  
                    local_data.risk_list = cal_risk(local_data.load_risk)
                    answer_all = ""
                    long_text = local_data.disease_info['정의'].values[0]
                    for m, risk_value in enumerate(local_data.risk_list):
                        local_data.risk_per = "NaN" if math.isnan(risk_value) else "Infinity" if risk_value == float('inf') else str(int(risk_value * 10000) / 100.0)
                        answer = f"\n{m+1}번 공식 결과, {local_data.disease_name}에 걸릴 확률은 {local_data.risk_per}%입니다."
                        answer_all += answer
                    answer_all += f"\n{long_text}" 
                    print(f"{local_data.addr} - 답변: {answer_all}\n")
                    conv_db.save_output_insert(local_data.addr, answer_all)
                    
                # DB에 해당 질병의 공식이 없을 경우  
                else:
                    print(f"{local_data.addr} - 예측 공식 없음\n")
                    answer = "해당 질병에 대한 예측 공식이 존재하지 않습니다."
                    print(f"{local_data.addr} - {answer}\n")
                    conv_db.save_output_update(local_data.addr, local_data.check_input_id, answer)  
                    
            elif local_data.user_intent in local_data.disease_info.columns:
                print(f"{local_data.addr} - 질병 의도 있음\n")

                # 의도가 증상인 경우
                if local_data.user_intent == "증상":
                    # DB에 증상과 증상설명 데이터가 둘 다 있을 경우
                    if local_data.disease_info[local_data.user_intent].values[0] is not None and local_data.disease_info['증상설명'].values[0] is not None:
                        long_text = local_data.disease_info[local_data.user_intent].values[0]
                        long_text += "\n\n"+local_data.disease_info['증상설명'].values[0]
                    # DB에 증상 데이터만 있을 경우
                    elif local_data.disease_info[local_data.user_intent].values[0] is not None and local_data.disease_info['증상설명'].values[0] is None:
                        long_text = local_data.disease_info[local_data.user_intent].values[0]
                    # DB에 증상설명 데이터만 있을 경우
                    elif local_data.disease_info[local_data.user_intent].values[0] is None and local_data.disease_info['증상설명'].values[0] is not None:
                        long_text = local_data.disease_info['증상설명'].values[0]
                    # DB에 증상과 증상설명 데이터가 둘 다 없을 경우
                    else:
                        long_text = f"{local_data.user_intent}에 대한 정보가 없습니다."
                        
                # 의도가 증상이 아닌 나머지 경우
                else:
                    # DB에 데이터가 있을 경우
                    if local_data.disease_info[local_data.user_intent].values[0] is not None:
                        long_text = local_data.disease_info[local_data.user_intent].values[0]
                    # DB에 데이터가 없을 경우
                    else:
                        long_text = f"{local_data.user_intent}에 대한 정보가 없습니다."

                print(f"{local_data.addr} - {local_data.user_intent}: {long_text}\n")
                conv_db.save_output_update(local_data.addr, local_data.check_input_id, long_text)

            elif local_data.user_intent == "수정":
                print(f"{local_data.addr} - 사용자 정보 수정\n")
                
                # 사용자 입력에 변수 이름이 있는지 확인
                question_db.connect()
                local_data.var_name = question_db.var_matching(local_data.query)  
                question_db.close()
                print(f"{local_data.addr} - 수정 변수: {local_data.var_name}")

                if local_data.var_name: 
                    process_convert_user_data(local_data.check_input_id, local_data.user_id, local_data.var_name, question_df)
                else:
                    answer = "변수명을 올바르게 입력해주세요."
                    print(f"{local_data.addr} - {answer}\n")
                    conv_db.save_output_update(local_data.addr, local_data.check_input_id, answer)
                    
            else:
                if local_data.search_term is not None and local_data.disease_info['정의'].values[0] is not None:
                    print(f"{local_data.addr} - 질병 의도 없음\n")
                    
                    long_text = local_data.disease_info['정의'].values[0]
                    print(f"{local_data.addr} - 정의: {long_text}\n")
                    conv_db.save_output_update(local_data.addr, local_data.check_input_id, long_text)
                
                else: 
                    print(f"{local_data.addr} - 일상 대화\n")
                    # 의도 파악
                    intent_predict = intent.predict_class(local_data.query)
                    intent_name = intent.labels[intent_predict]
                    # 개체명 파악
                    ner_predicts = ner.predict(local_data.query)
                    ner_tags = ner.predict_tags(local_data.query)

                    try:
                        f = FindAnswer(chat_db)    
                        answer_text = f.search(intent_name, ner_tags)
                        answer_chat = f.tag_to_word(ner_predicts, answer_text[3])
                    except:
                        answer_chat = "죄송해요 무슨 말인지 모르겠어요. 조금 더 공부 할게요."
                    print(f"{local_data.addr} - {answer_chat}\n")
                    conv_db.save_output_update(local_data.addr, local_data.check_input_id, answer_chat)
    
    except Exception as e:
        print(f"{local_data.addr} - {e}")

    finally:
        chat_db.close()
        conv_db.reset_user_table(local_data.addr)
        conv_db.close()
        
        if local_data.connect:
            local_data.connect.close()
        print(f"{local_data.addr} 클라이언트 종료 완료")

## 챗봇 실행

In [ ]:
if __name__ == '__main__':

    # 질문/답변 학습 디비 연결 객체 생성
    chat_db = SQliteDB.chat_DB('../chatbot_db/chat.db')
    #예측 공식 추가 디비 연결
    formula_db = SQliteDB.formula_DB('../chatbot_db/formula.db')
    # 사용자 정보 디비 연결
    user_db = SQliteDB.user_DB('../chatbot_db/user.db')
    # 변수 질문 정보 디비 연결
    question_db = SQliteDB.question_DB('../chatbot_db/question.db')
    # 질병 정보 디비 연결
    disease_db = SQliteDB.disease_DB('../chatbot_db/disease.db')
    # 대화 디비 연결
    conv_db = SQliteDB.conv_DB('../chatbot_db/conversation.db')
    # 의도 데이터 형태소 분석
    intents = UserIntent()
    intents_result = intents.intents_result()
    
    print("DB 접속")

    # 봇 서버 동작
    port = 5050   # 챗봇 엔진 서버의 통신 포트
    listen = 100   # 최대 클라이언트 연결 수(최대 동시 접속자 수)

    bot = BotServer(port, listen)
    bot.create_sock()
    print("bot start")

    local_data = threading.local()
    lock = threading.Lock()
    # 기존 연결된 클라이언트의 IP 주소를 저장하는 딕셔너리
    client_sockets = {}
    
    while True:
        # 클라이언트 접속 대기
        connect, address = bot.ready_for_client() # 클라이언트로부터 연결 수락
        print('===========================')

        user_ip = connect.recv(1024).decode('utf-8')
        addr = (user_ip, address[1])
        print(f"새로운 연결: {addr}")

        # 이전에 동일한 IP 주소로 연결된 소켓이 있다면 해당 소켓을 닫음
        if addr[0] in client_sockets:
            client_sockets[addr[0]].close()
            print(f"Closed previous connection from {addr[0]}\n")
        # 연결된 클라이언트의 소켓 정보를 저장
        client_sockets[addr[0]] = connect

        # to_client 스레드 시작 
        params = {
            "chat_db": chat_db,
            "user_db": user_db,
            "formula_db": formula_db,
            "question_db": question_db,
            "disease_db": disease_db,
            "conv_db": conv_db,
        }
        client = threading.Thread(target=to_client, args=(
            connect,     # 클라이언트 연결 소켓
            addr[0],     # 클라이언트 연결 주소 정보
            params,    # 스레드 함수 파라미터
            intents_result, # intents dict 형태소 분석 결과
        ))
        client.start()  # 스레드 시작     

DB 접속
bot start
새로운 연결: ('192.168.1.106', 61924)
Completed create table from 192_168_1_106

192.168.1.106 - 안녕하세요, 챗봇입니다. 어떤 도움이 필요하신가요?
192.168.1.106 - 사용자 ID를 입력하세요.
새로운 연결: ('192.168.1.118', 61963)
Completed create table from 192_168_1_118

192.168.1.118 - 안녕하세요, 챗봇입니다. 어떤 도움이 필요하신가요?
192.168.1.118 - 사용자 ID를 입력하세요.
새로운 연결: ('192.168.1.108', 61977)
Completed create table from 192_168_1_108

192.168.1.108 - 안녕하세요, 챗봇입니다. 어떤 도움이 필요하신가요?
192.168.1.108 - 사용자 ID를 입력하세요.
새로운 연결: ('192.168.1.119', 61993)
Completed create table from 192_168_1_119

192.168.1.119 - 안녕하세요, 챗봇입니다. 어떤 도움이 필요하신가요?
192.168.1.119 - 사용자 ID를 입력하세요.
192.168.1.108 - user ID: ddd

192.168.1.108 - 등록되지 않은 사용자입니다.
192.168.1.108 - 사용자 ID를 입력하세요.
192.168.1.118 - user ID: test1

192.168.1.118 - ID 확인 완료
192.168.1.118 - 질문을 입력하세요.
192.168.1.119 - user ID: test5

192.168.1.119 - ID 확인 완료
192.168.1.119 - 질문을 입력하세요.
192.168.1.119 - query: 배달언제와?

192.168.1.119 - search_term: None
192.168.1.119 - user_intent: None

192.168.1.119 -